# CVPR Project 2: Detecting Lines and Circles

All projects in this course will be done with these iPython notebooks. These are convenient ways for you to quickly and easily interact with the code. A notebook contains many blocks of code, each of which can be run independently. You can run a cell with ctrl+enter or shift+enter (to move to the next cell).

If there are any notes you wish to leave for me as I test your code, you may leave them here.


Notes: You must use a photo taken by you containing a few circles and lines, and test your code on that custom file. 
Save the file as final.jpg. I have given a dummpy final.jpg file, that you should replace with your own file. 


## Setup

Both functions you need to implement in this project can be found in student_code.py. Start by implementing hough_lines_vote_acc and then hough_circles_vote_acc. Implement more efficient verison of find_circles for extra credit. 

In [ ]:
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import time

%matplotlib inline
from utils import  hough_peaks, hough_lines_draw, hough_circles_draw, preprocess, plotFigure, plotFigureGray
# from student_code

from student_code import hough_lines_vote_acc, hough_circles_vote_acc, find_circles
%load_ext autoreload
%autoreload 2


# load_image loads image as an BGR image.
# The resultant array data-type is 'uint8' numpy array

image0 = cv2.imread('../data/input0.png')
image0noise = cv2.imread('../data/input0-noise.png')
image1 = cv2.imread('../data/input1.png')
image2 = cv2.imread('../data/input2.png')
image3 = cv2.imread('../data/input3.png')


plotFigure(image0)
plotFigure(image0noise)
plotFigure(image1)
plotFigure(image2)
plotFigure(image3)


## Preprosseing. 
Feel Free to play with the arguments. 

In [ ]:

edge_img0 = preprocess(image0, 1, 1, 1, 40, 80) 
edge_img0noise = preprocess(image0noise, 1, 1, 1, 800, 800)  
edge_img1 = preprocess(image1, 3, 11, 3, 20, 40)  
edge_img2 = preprocess(image2, 5, 7, 5, 50, 100)  
edge_img3 = preprocess(image3, 6, 7, 5, 20, 40)  

In [ ]:
# display the images
plotFigureGray(edge_img0)
plotFigureGray(edge_img0noise)
plotFigureGray(edge_img1)
plotFigureGray(edge_img2)
plotFigureGray(edge_img3)


### Checkerboard Line Detection

In [ ]:


start_time = time.time()
H, thetas, rhos = hough_lines_vote_acc(edge_img0)
cv2.imwrite('../results/img0HS.png', H)

# Detect peaks on the hough space of the edge image and highlight them
peaks = hough_peaks(H, numpeaks=10, threshold=100, nhood_size=50)

H_peaks = H.copy()
for peak in peaks:
    cv2.circle(H_peaks, tuple(peak[::-1]), 8, (255,255,255), 1)
cv2.imwrite('../results/img0HSHL.png', H_peaks)

# Draw the detected lines on the image

img_lines= image0.copy()
hough_lines_draw(img_lines, '../results/img0line.png', peaks, rhos, thetas)
print('Time elapsed: %.2f s'%(time.time()-start_time))

### Checkerboard Line Detection (with noise)

In [ ]:
start_time = time.time()


H, thetas, rhos = hough_lines_vote_acc(edge_img0noise)
cv2.imwrite('../results/image0noiseHS.png', H)

# Detect peaks on the hough space of the edge image and highlight them
peaks = hough_peaks(H, numpeaks=10, threshold=100, nhood_size=50)
H_peaks = H.copy()
for peak in peaks:
    cv2.circle(H_peaks, tuple(peak[::-1]), 5, (255,255,255), -1)
cv2.imwrite('../results/img0noiseHSHL.png', H_peaks)



# Draw the detected lines on the image

img_lines= image0noise.copy()
hough_lines_draw(img_lines, '../results/img0noiseline.png', peaks, rhos, thetas)
print('Time elapsed: %.2f s'%(time.time()-start_time))

In [ ]:
edge_img1 = preprocess(image1, 3, 11, 3, 30, 40)  
plotFigureGray(edge_img1)

start_time = time.time()
H, thetas, rhos = hough_lines_vote_acc(edge_img1)
cv2.imwrite('../results/image1HS.png', H)

# Detect peaks on the hough space of the edge image and highlight them
peaks = hough_peaks(H, numpeaks=10, threshold=100, nhood_size=50)
H_peaks = H.copy()
for peak in peaks:
    cv2.circle(H_peaks, tuple(peak[::-1]), 5, (255,255,255), -1)

cv2.imwrite('../results/img1HSHL.png', H_peaks)


# Draw the detected lines on the image

img_lines= image1.copy()
hough_lines_draw(img_lines, '../results/img1line.png', peaks, rhos, thetas)
print('Time elapsed: %.2f s'%(time.time()-start_time))

## Hough Circle Detector (fixed Radius)


In [ ]:
edge_img1 = preprocess(image1, 3, 11, 3, 30, 40)  

start_time = time.time()

# detect circles with radius = 20 and save image
H_20 = hough_circles_vote_acc(edge_img1, 20)
cv2.imwrite('../results/image1CircleHS.png', H_20)

peaks = hough_peaks(H_20, numpeaks=10, threshold=140, nhood_size=100)
img_circles = image1.copy()
img_circles = hough_circles_draw(img_circles, '../results/img1circle20.png', peaks, 20)

print('Time elapsed: %.2f s'%(time.time()-start_time))

## Hough Line and Circle Detector (Radius Range)

In [ ]:
edge_img1 = preprocess(image1, 3, 11, 3, 20, 40)  
start_time = time.time()


H, thetas, rhos = hough_lines_vote_acc(edge_img1)
# Detect peaks on the hough space of the edge image and highlight them
peaks = hough_peaks(H, numpeaks=10, threshold=100, nhood_size=50)
H_peaks = H.copy()
for peak in peaks:
    cv2.circle(H_peaks, tuple(peak[::-1]), 5, (255,255,255), -1)

centers, radii = find_circles(edge_img1, [15, 40], threshold=108, nhood_size=100)
img_circles = image1.copy()
for i in range(len(radii)):
    img_circles = hough_circles_draw(img_circles, '../results/img1circle20-40.png',
                                     centers[i], radii[i])

# Draw the detected lines on the image
hough_lines_draw(img_circles, '../results/img1lineAndCircle.png', peaks, rhos, thetas)

print('Time elapsed: %.2f'%(time.time()-start_time))


### PlayZone. Play with any image of your choice. 

In [ ]:
img = cv2.imread('../data/final.jpg')

plotFigure(img)

edge_img = preprocess(img, 3, 11, 3, 20, 40)  
start_time = time.time()


H, thetas, rhos = hough_lines_vote_acc(edge_img)
cv2.imwrite('../results/finalHS.png', H)

# Detect peaks on the hough space of the edge image and highlight them
peaks = hough_peaks(H, numpeaks=10, threshold=100, nhood_size=50)
H_peaks = H.copy()
for peak in peaks:
    cv2.circle(H_peaks, tuple(peak[::-1]), 5, (255,255,255), -1)

centers, radii = find_circles(edge_img, [15, 40], threshold=108, nhood_size=100)
img_circles = img.copy()
for i in range(len(radii)):
    img_circles = hough_circles_draw(img_circles, '../results/finalCircles.png',
                                     centers[i], radii[i])

# Draw the detected lines on the image
hough_lines_draw(img_circles, '../results/finalLinesAndCircles.png', peaks, rhos, thetas)

print('Time elapsed: %.2f'%(time.time()-start_time))
